Cuidado com o gerenciador de contexto do PyODBC: ele não fecha conexões.

[Link](https://github.com/mkleehammer/pyodbc/issues/43#issuecomment-1046678648)

In [1]:
import warnings
import pandas as pd
from decouple import config
from db_manager import DBManager

DRIVER = config("DRIVER")
DATABASE = config("DATABASE")
USERNAME = config("USERNAME")
PASSWORD = config("PASSWORD")
HOST = config("HOST")
PORT = config("PORT")

db = DBManager(
    driver=DRIVER,
    database=DATABASE,
    username=USERNAME,
    password=PASSWORD,
    host=HOST,
    port=PORT,
)

query_all_tables = """
SELECT table_name FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog') AND
table_type = 'BASE TABLE';
"""

query_us_column_names = """
SELECT column_name FROM information_schema.columns
WHERE table_schema = 'public'
AND table_name = 'us_countries_pop_est_2019'
ORDER BY ordinal_position;
"""

with db:
    tables = [item[0] for item in db.query(query_all_tables)]
    query_us_first_5 = db.query("SELECT * FROM us_countries_pop_est_2019 LIMIT 5")
    query_us_columns = [item[0] for item in db.query(query_us_column_names)]

    # beware of the warning supression!! Pandas was only tested with SQLAlchemy
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        all_tables_as_dataframes = {
            table: pd.read_sql_query(f"SELECT * FROM {table}", db.connection)
            for table in tables
        }

    # get all data without Pandas
    db.execute("SELECT * FROM us_countries_pop_est_2019")
    data_pop_est_2019 = db.fetchall()


In [2]:
print(db)

{PostgreSQL Unicode} for analysis on 5432


In [3]:
repr(db)

"DBManager('analysis', '{PostgreSQL Unicode}', 'chicolucio', '<hidden>', '172.20.0.3', '5432')"

In [4]:
tables

['teachers', 'us_countries_pop_est_2019']

In [5]:
query_us_columns

['state_fips',
 'county_fips',
 'region',
 'state_name',
 'county_name',
 'area_land',
 'area_water',
 'internal_point_lat',
 'internal_point_lon',
 'pop_est_2018',
 'pop_est_2019',
 'births_2019',
 'deaths_2019',
 'international_migr_2019',
 'domestic_migr_2019',
 'residual_2019']

In [6]:
query_us_first_5

[('01', '001', 3, 'Alabama', 'Autauga County', 1539602137, 25706961, Decimal('32.5322367'), Decimal('-86.6464395'), 55533, 55869, 624, 541, -16, 270, -1),
 ('01', '003', 3, 'Alabama', 'Baldwin County', 4117621645, 1132980868, Decimal('30.6592183'), Decimal('-87.7460666'), 217855, 223234, 2304, 2326, 80, 5297, 24),
 ('01', '005', 3, 'Alabama', 'Barbour County', 2292160139, 50523213, Decimal('31.8702531'), Decimal('-85.4051035'), 24872, 24686, 256, 312, 13, -141, -2),
 ('01', '007', 3, 'Alabama', 'Bibb County', 1612167476, 9602089, Decimal('33.0158929'), Decimal('-87.1271475'), 22367, 22394, 240, 252, 10, 31, -2),
 ('01', '009', 3, 'Alabama', 'Blount County', 1670103866, 15015467, Decimal('33.9773575'), Decimal('-86.5664400'), 57771, 57826, 651, 657, 6, 59, -4)]

In [7]:
all_tables_as_dataframes.keys()

dict_keys(['teachers', 'us_countries_pop_est_2019'])

In [8]:
all_tables_as_dataframes["teachers"]

,id,first_name,last_name,school,hire_date,salary
0,1,Janet,Smith,F.D. Roosevelt HS,2011-10-30,36200.0
1,2,Lee,Reynolds,F.D. Roosevelt HS,1993-05-22,65000.0
2,3,Samuel,Cole,Myers Middle School,2005-08-01,43500.0
3,4,Samantha,Bush,Myers Middle School,2011-10-30,36200.0
4,5,Betty,Diaz,Myers Middle School,2005-08-30,43500.0
5,6,Kathleen,Roush,F.D. Roosevelt HS,2010-10-22,38500.0


In [9]:
all_tables_as_dataframes["us_countries_pop_est_2019"].head()

,state_fips,county_fips,region,state_name,county_name,area_land,area_water,internal_point_lat,internal_point_lon,pop_est_2018,pop_est_2019,births_2019,deaths_2019,international_migr_2019,domestic_migr_2019,residual_2019
0,01,001,3,Alabama,Autauga County,1539602137,25706961,32.532237,-86.646439,55533,55869,624,541,-16,270,-1
1,01,003,3,Alabama,Baldwin County,4117621645,1132980868,30.659218,-87.746067,217855,223234,2304,2326,80,5297,24
2,01,005,3,Alabama,Barbour County,2292160139,50523213,31.870253,-85.405103,24872,24686,256,312,13,-141,-2
3,01,007,3,Alabama,Bibb County,1612167476,9602089,33.015893,-87.127148,22367,22394,240,252,10,31,-2
4,01,009,3,Alabama,Blount County,1670103866,15015467,33.977357,-86.566440,57771,57826,651,657,6,59,-4


In [10]:
df_us_pop_est_2019 = pd.DataFrame.from_records(
    data_pop_est_2019, columns=query_us_columns
)
df_us_pop_est_2019.head()


,state_fips,county_fips,region,state_name,county_name,area_land,area_water,internal_point_lat,internal_point_lon,pop_est_2018,pop_est_2019,births_2019,deaths_2019,international_migr_2019,domestic_migr_2019,residual_2019
0,01,001,3,Alabama,Autauga County,1539602137,25706961,32.5322367,-86.6464395,55533,55869,624,541,-16,270,-1
1,01,003,3,Alabama,Baldwin County,4117621645,1132980868,30.6592183,-87.7460666,217855,223234,2304,2326,80,5297,24
2,01,005,3,Alabama,Barbour County,2292160139,50523213,31.8702531,-85.4051035,24872,24686,256,312,13,-141,-2
3,01,007,3,Alabama,Bibb County,1612167476,9602089,33.0158929,-87.1271475,22367,22394,240,252,10,31,-2
4,01,009,3,Alabama,Blount County,1670103866,15015467,33.9773575,-86.5664400,57771,57826,651,657,6,59,-4
